In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# knowing the data

In [ ]:
df=pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")

df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe()

# feature selection

In [ ]:
df.corr

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(20,20))
sns.heatmap(df.corr(),cmap="Reds",annot=True)
plt.show()

from the heatmap, none of the colums is correlated,hence the only column that will be drop is the ID column as it will have no contribution to the machine learning algorithm

In [ ]:
df.drop(columns=["id"],inplace=True)

Data Visualisaion by creating boxplot

In [ ]:
fig=plt.figure(figsize=(20,20))
for (i,column) in zip(np.arange(1,21,1),df.columns[:-1]):
    sns.boxplot(data=df[column], ax=fig.add_subplot(4,5,i,title=column))

In [ ]:
# def outlier_detection(df, column):
#     outlier=[]
#     for x in df[column]:
#         z=np.divide( x-np.mean(df[column]), np.std(df[column]))
#         if z > 3:
#             outlier.append(x)
#             df[column]=df[column].replace(x, np.mean(df[column])).copy()
#     return column,outlier        
    
            

In [ ]:
# for column in df.columns[:-1]:
#     outlier_detection(df,column) 

spliting the data into dependend(y) and indepedent variable(X)

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

X=df.iloc[:,:-1]
scalar=MinMaxScaler()
X_scaled= scalar.fit_transform(X)

X=pd.DataFrame(data=X_scaled, columns=X.columns,index=X.index)
y=df.iloc[:,-1:]
y

Choosen a model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge,Lasso,LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score,mean_squared_error

In [ ]:
# param_g= {'alpha':[0.001,0.01,0.1,2.0,4.0,16.0,100]}
# Gridsearch=GridSearchCV(estimator=Ridge(),param_grid=param_g,scoring="r2")
# Gridsearch.fit(X,y)
# Gridsearch.best_params_

In [ ]:
param_d= {"max_depth":[2,3],"n_estimators":[50,100],
          "min_samples_split":[2,4 ],"max_features":["sqrt",None],
         "criterion":["squared_error","absolute_error"]}
Rdfr=RandomForestRegressor(max_samples=3,oob_score=True,random_state=2, n_jobs=-1) 
Randomsearch=RandomizedSearchCV(estimator=Rdfr,param_distributions=param_d, n_iter=5, scoring="neg_mean_squared_error",random_state=1, cv=4,n_jobs=-1)
Randomsearch.fit(X,np.ravel(y,order="F"))
Randomsearch.cv_results_

df_result= pd.DataFrame(Randomsearch.cv_results_)
df_result
# Randomsearch.best_par
# Randomsearch.best_score_

In [ ]:
# param_g= {"max_depth":[2,3],"n_estimators":[100,400],
#           "min_samples_split":[2,4 ],"max_features":["sqrt","None"],
#          "criterion":["squared_error","absolute_error"]}
# Rdfr=RandomForestRegressor(max_samples=0.7,oob_score=True) 
# Gridsearch=GridSearchCV(Rdfr,param_grid=param_g,scoring="r2", cv=3,n_jobs=-1)
# Gridsearch.fit(X,np.ravel(y, order="F"))
# Gridsearch.best_params_
# Gridsearch.best_score_

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
# model=Ridge(alpha=16)
# model.fit(X_train,y_train)
# y_pred=model.predict(X_test)
# score=r2_score(y_test,y_pred)
# score